In [64]:
import sys
import numpy as np
import time
import matplotlib.pyplot as plt
from scipy import interpolate 
sys.path.append(r'C:\Chuji\Code_and_Data\MyCode')

import Circuit_Simulator 
import PulseGenerator as PG
from toolfunc import *
from toolfunc.adia_analysis import *
from toolfunc.pulse_filter import *
import scipy.optimize as sci_op
from toolfunc import DE
from Circuit_Simulator.Qobj import *

In [65]:
gate_time=30e-9
SRATE=10e9
f_term=1
lamb1 = 0.86256836
total_len = gate_time + 8e-9

Seq=PG.Sequence(total_len=total_len,sample_rate=SRATE,complex_trace=False)
Seq.clear_pulse(tips_on=False)
Seq.add_pulse('Adiabatic',t0=total_len/2,width=gate_time,plateau=0e-9,frequency=0,F_Terms=f_term,Lcoeff=np.array(lamb1),Q1_freq=6.0e9,
             CPLR_idle_freq=7.87e9,Q2_freq=5.4e9,constant_coupling=False,r1c=0.018,r2c=0.018,r12=0.0015,anhar_CPLR=-300e6,
             anhar_Q1=-250e6,anhar_Q2=-250e6,negative_amplitude=False,dfdV=7.87e9-1e9,gap_threshold=8e6,freqpoints=301,pulsepoints=601)
Seq.add_filter('Gauss Low Pass',300e6)
flux_pulse=Seq.get_sequence()

raw_config = Circuit_Simulator.RawConfig(qubit_num=5,dimension=3,circuit_type=1,initial_state='-Z+Z+Z+Z+Z',sampling_rate=SRATE)
raw_config.load_default_value(modulation=True,decoherence=False,use_capacitance=False)

raw_config.setValue('Q1 f01_max',6.0e9)
raw_config.setValue('Q2 f01_max',7.87e9)
raw_config.setValue('Q3 f01_max',5.4e9)
raw_config.setValue('Q4 f01_max',8.0e9)
raw_config.setValue('Q5 f01_max',6.1e9)

raw_config.setValue('Q1 f01_min',1e9)
raw_config.setValue('Q2 f01_min',1e9)
raw_config.setValue('Q3 f01_min',1e9)
raw_config.setValue('Q4 f01_min',1e9)
raw_config.setValue('Q5 f01_min',1e9)    

raw_config.setValue('Q1 Ec',0.25e9)
raw_config.setValue('Q2 Ec',0.3e9)
raw_config.setValue('Q3 Ec',0.25e9)
raw_config.setValue('Q4 Ec',0.30e9)
raw_config.setValue('Q5 Ec',0.25e9)

raw_config.setValue('r12',0.018)
raw_config.setValue('r23',0.018)
raw_config.setValue('r13',0.0015)
raw_config.setValue('r34',0.018)
raw_config.setValue('r45',0.018)
raw_config.setValue('r35',0.0015)

raw_config.setValue('r14',0.0004)
raw_config.setValue('r15',0.0004)
raw_config.setValue('r24',0.0000)
raw_config.setValue('r25',0.0000)


raw_config.setValue('Q2 Voltage period',-1)
raw_config.setValue('Q2 Voltage operating point',0)
raw_config.setValue('Q2 Flux',flux_pulse)

simu_config = Circuit_Simulator.read_config(raw_config.get_dict())
Simulator = Circuit_Simulator.Simulator(simu_config)
Simulator.show_pulse()

# Simulator.performsimulation(solver_type=1)
# Simulator.UnitaryResult.get_U(-1)

# Simulator.UnitaryResult.get_subspace_operator(['00000','00001','00100','00101','10000','10001','10100','10101'])
# diag_value = Simulator.UnitaryResult.U_sub.diag()
# phaseQ3=np.angle(diag_value[1])-np.angle(diag_value[0])

# Simulator.UnitaryResult.set_Target_gate('CZ_I')
# Simulator.UnitaryResult.remove_single_qubit_gate()
# Simulator.UnitaryResult.get_Gate_Fidelity()
# Simulator.UnitaryResult.U_sub 
# fidelity = Simulator.UnitaryResult.Gate_Fidelity

{'Q1_freq': '6000.0MHz', 'Q1_anhar': '-125.0MHz', 'Q2_freq': '7870.0MHz', 'Q2_anhar': '-150.0MHz', 'Q3_freq': '5400.0MHz', 'Q3_anhar': '-125.0MHz', 'Q4_freq': '8000.0MHz', 'Q4_anhar': '-150.0MHz', 'Q5_freq': '6100.0MHz', 'Q5_anhar': '-125.0MHz', 'g12': '123.6903MHz', 'g13': '8.5381MHz', 'g14': '2.7713MHz', 'g15': '2.4199MHz', 'g23': '117.3429MHz', 'g24': '0.0MHz', 'g25': '0.0MHz', 'g34': '118.3081MHz', 'g35': '8.609MHz', 'g45': '125.7426MHz'}


In [66]:
def cost_func(pulse_params,*args):
    gate_time,SRATE,r_v1,r_v2,r_v3,r_v4=args
    
    lamb1 = pulse_params
    total_len = gate_time + 6e-9
    
    Seq=PG.Sequence(total_len=total_len,sample_rate=SRATE,complex_trace=False)
    Seq.clear_pulse(tips_on=False)
    Seq.add_pulse('Adiabatic',t0=total_len/2,width=gate_time,plateau=0e-9,frequency=0,F_Terms=1,Lcoeff=np.array(lamb1),min_C=1e9,down_tuning=True,Q1_freq=6.0e9,
                 CPLR_idle_freq=7.87e9,Q2_freq=5.4e9,constant_coupling=False,r1c=0.018,r2c=0.018,r12=0.0015,anhar_CPLR=-300e6,
                 anhar_Q1=-250e6,anhar_Q2=-250e6,negative_amplitude=False,dfdV=7.87e9-1e9,gap_threshold=8e6,freqpoints=301,pulsepoints=601)
    Seq.add_filter('Gauss Low Pass',300e6)
    flux_pulse=Seq.get_sequence()

    raw_config = Circuit_Simulator.RawConfig(qubit_num=5,dimension=3,circuit_type=1,initial_state='-Z+Z+Z+Z+Z',sampling_rate=SRATE)
    raw_config.load_default_value(modulation=True,decoherence=False,use_capacitance=False)

    raw_config.setValue('Q1 f01_max',6.0e9)
    raw_config.setValue('Q2 f01_max',7.87e9)
    raw_config.setValue('Q3 f01_max',5.4e9)
    raw_config.setValue('Q4 f01_max',8.22e9)
    raw_config.setValue('Q5 f01_max',6.1e9)

    raw_config.setValue('Q1 f01_min',1e9)
    raw_config.setValue('Q2 f01_min',1e9)
    raw_config.setValue('Q3 f01_min',1e9)
    raw_config.setValue('Q4 f01_min',1e9)
    raw_config.setValue('Q5 f01_min',1e9)    

    raw_config.setValue('Q1 Ec',0.25e9)
    raw_config.setValue('Q2 Ec',0.30e9)
    raw_config.setValue('Q3 Ec',0.25e9)
    raw_config.setValue('Q4 Ec',0.30e9)
    raw_config.setValue('Q5 Ec',0.25e9)

    raw_config.setValue('r12',0.018)
    raw_config.setValue('r23',0.018)
    raw_config.setValue('r13',0.0015)
    raw_config.setValue('r34',0.018)
    raw_config.setValue('r45',0.018)
    raw_config.setValue('r35',0.0015)

    raw_config.setValue('r14',r_v1)
    raw_config.setValue('r15',r_v2)
    raw_config.setValue('r24',r_v3)
    raw_config.setValue('r25',r_v4)
    
    raw_config.setValue('Q2 Voltage period',-1)
    raw_config.setValue('Q2 Voltage operating point',0)
    raw_config.setValue('Q2 Flux',flux_pulse)

    simu_config = Circuit_Simulator.read_config(raw_config.get_dict())
    Simulator = Circuit_Simulator.Simulator(simu_config)

    Simulator.performsimulation(solver_type=1)
    Simulator.UnitaryResult.get_U(-1)

    Simulator.UnitaryResult.get_subspace_operator(['00000','00001','00100','00101','10000','10001','10100','10101'])
#     diag_value = Simulator.UnitaryResult.U_sub.diag()
#     phaseQ3=np.angle(diag_value[1])-np.angle(diag_value[0])
    Simulator.UnitaryResult.set_Target_gate('CZ_I')
    Simulator.UnitaryResult.remove_single_qubit_gate()
    Simulator.UnitaryResult.get_Gate_Fidelity()
    Simulator.UnitaryResult.U_sub 
    fidelity = Simulator.UnitaryResult.Gate_Fidelity
    print('lamb1:',lamb1)
    print('error:',1-fidelity)
    return 1 - fidelity

def cost_func_C23(pulse_params,*args):
    gate_time,SRATE,r_v1,r_v2,r_v3,r_v4=args
    
    lamb1 = pulse_params
    total_len = gate_time + 6e-9
    
    Seq=PG.Sequence(total_len=total_len,sample_rate=SRATE,complex_trace=False)
    Seq.clear_pulse(tips_on=False)
    Seq.add_pulse('Adiabatic',t0=total_len/2,width=gate_time,plateau=0e-9,frequency=0,F_Terms=1,Lcoeff=np.array(lamb1),min_C=1e9,down_tuning=True,Q1_freq=6.1e9,
                 CPLR_idle_freq=8.22e9,Q2_freq=5.4e9,constant_coupling=False,r1c=0.018,r2c=0.018,r12=0.0015,anhar_CPLR=-300e6,
                 anhar_Q1=-250e6,anhar_Q2=-250e6,negative_amplitude=False,dfdV=8.22e9-1e9,gap_threshold=8e6,freqpoints=301,pulsepoints=601)
    Seq.add_filter('Gauss Low Pass',300e6)
    flux_pulse=Seq.get_sequence()

    raw_config = Circuit_Simulator.RawConfig(qubit_num=5,dimension=3,circuit_type=1,initial_state='-Z+Z+Z+Z+Z',sampling_rate=SRATE)
    raw_config.load_default_value(modulation=True,decoherence=False,use_capacitance=False)

    raw_config.setValue('Q1 f01_max',6.0e9)
    raw_config.setValue('Q2 f01_max',7.87e9)
    raw_config.setValue('Q3 f01_max',5.4e9)
    raw_config.setValue('Q4 f01_max',8.22e9)
    raw_config.setValue('Q5 f01_max',6.1e9)

    raw_config.setValue('Q1 f01_min',1e9)
    raw_config.setValue('Q2 f01_min',1e9)
    raw_config.setValue('Q3 f01_min',1e9)
    raw_config.setValue('Q4 f01_min',1e9)
    raw_config.setValue('Q5 f01_min',1e9)    

    raw_config.setValue('Q1 Ec',0.25e9)
    raw_config.setValue('Q2 Ec',0.30e9)
    raw_config.setValue('Q3 Ec',0.25e9)
    raw_config.setValue('Q4 Ec',0.30e9)
    raw_config.setValue('Q5 Ec',0.25e9)

    raw_config.setValue('r12',0.018)
    raw_config.setValue('r23',0.018)
    raw_config.setValue('r13',0.0015)
    raw_config.setValue('r34',0.018)
    raw_config.setValue('r45',0.018)
    raw_config.setValue('r35',0.0015)

    raw_config.setValue('r14',r_v1)
    raw_config.setValue('r15',r_v2)
    raw_config.setValue('r24',r_v3)
    raw_config.setValue('r25',r_v4)
    
    raw_config.setValue('Q4 Voltage period',-1)
    raw_config.setValue('Q4 Voltage operating point',0)
    raw_config.setValue('Q4 Flux',flux_pulse)

    simu_config = Circuit_Simulator.read_config(raw_config.get_dict())
    Simulator = Circuit_Simulator.Simulator(simu_config)

    Simulator.performsimulation(solver_type=1)
    Simulator.UnitaryResult.get_U(-1)

    Simulator.UnitaryResult.get_subspace_operator(['00000','00001','00100','00101','10000','10001','10100','10101'])
#     diag_value = Simulator.UnitaryResult.U_sub.diag()
#     phaseQ3=np.angle(diag_value[1])-np.angle(diag_value[0])
    Simulator.UnitaryResult.set_Target_gate('I_CZ')
    Simulator.UnitaryResult.remove_single_qubit_gate()
    Simulator.UnitaryResult.get_Gate_Fidelity()
    Simulator.UnitaryResult.U_sub 
    fidelity = Simulator.UnitaryResult.Gate_Fidelity
    print('lamb1:',lamb1)
    print('error:',1-fidelity)
    return 1 - fidelity

In [40]:
# error_r14=[]
# for r_v1 in np.linspace(0.00000,0.0001,11):
#     error_r14.append( cost_func(0.8575,*(30e-9,10e9,r_v1,0,0,0)))

error_r15=[]
for r_v2 in np.linspace(0.00000,0.0001,11):
    error_r15.append( cost_func(0.8575,*(30e-9,10e9,0,r_v2,0,0)))
    
# error_r24=[]
# for r_v3 in np.linspace(0.00000,0.0001,11):
#     error_r24.append( cost_func(0.8575,*(30e-9,10e9,0,0,r_v3,0)))
    
# error_r25=[]
# for r_v4 in np.linspace(0.00000,0.0001,11):
#     error_r25.append( cost_func(0.8575,*(30e-9,10e9,0,0,0,r_v4)))

KeyboardInterrupt: 

In [ ]:
# np.savetxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_CZI_r14.txt',error_r14)
# np.savetxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_CZI_r15.txt',error_r15)
# np.savetxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_CZI_r24.txt',error_r24)
# np.savetxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_CZI_r25.txt',error_r25)

## plot data

In [67]:
np.linspace(0.00000,0.0001,11)

array([0.e+00, 1.e-05, 2.e-05, 3.e-05, 4.e-05, 5.e-05, 6.e-05, 7.e-05,
       8.e-05, 9.e-05, 1.e-04])

In [68]:
SRATE=10e9
g14_arr=[]
g15_arr=[]
g24_arr=[]
g25_arr=[]
for r_v in np.linspace(0.00000,0.0004,21):
    raw_config = Circuit_Simulator.RawConfig(qubit_num=5,dimension=3,circuit_type=1,initial_state='-Z+Z+Z+Z+Z',sampling_rate=SRATE)
    raw_config.load_default_value(modulation=True,decoherence=False,use_capacitance=False)

    raw_config.setValue('Q1 f01_max',6.0e9)
    raw_config.setValue('Q2 f01_max',7.87e9)
    raw_config.setValue('Q3 f01_max',5.4e9)
    raw_config.setValue('Q4 f01_max',8.22e9)
    raw_config.setValue('Q5 f01_max',6.1e9)

    raw_config.setValue('Q1 f01_min',1e9)
    raw_config.setValue('Q2 f01_min',1e9)
    raw_config.setValue('Q3 f01_min',1e9)
    raw_config.setValue('Q4 f01_min',1e9)
    raw_config.setValue('Q5 f01_min',1e9)    

    raw_config.setValue('Q1 Ec',0.25e9)
    raw_config.setValue('Q2 Ec',0.30e9)
    raw_config.setValue('Q3 Ec',0.25e9)
    raw_config.setValue('Q4 Ec',0.30e9)
    raw_config.setValue('Q5 Ec',0.25e9)

    raw_config.setValue('r12',0.018)
    raw_config.setValue('r23',0.018)
    raw_config.setValue('r13',0.0015)
    raw_config.setValue('r34',0.018)
    raw_config.setValue('r45',0.018)
    raw_config.setValue('r35',0.0015)

    raw_config.setValue('r14',r_v)
    raw_config.setValue('r15',r_v)
    raw_config.setValue('r24',r_v)
    raw_config.setValue('r25',r_v)

    raw_config.setValue('Q4 Voltage period',-1)
    raw_config.setValue('Q4 Voltage operating point',0)
    raw_config.setValue('Q4 Flux',flux_pulse)

    simu_config = Circuit_Simulator.read_config(raw_config.get_dict())
    Simulator = Circuit_Simulator.Simulator(simu_config)

    g14_arr.append( simu_config['Channels']['Channel18']['value'][0]/1e6 )
    g15_arr.append( simu_config['Channels']['Channel19']['value']/1e6 )
    g24_arr.append( simu_config['Channels']['Channel21']['value'][0]/1e6 )
    g25_arr.append( simu_config['Channels']['Channel22']['value']/1e6 )

In [49]:
# g14 g15 g24 g25

In [72]:
# fig=plt.figure()
# ax = fig.add_subplot(111)
# ax.plot(np.linspace(0.00000,0.0001,11),error_CZI_r14,'g','-.')
# ax.plot(np.linspace(0.00000,0.0001,11),error_r14,'g''--')

# ax.plot(np.linspace(0.00000,0.0004,21),error_CZI_r15,'r')
# ax.plot(np.linspace(0.00000,0.0004,21),error_r15,'r')

# ax.plot(np.linspace(0.00000,0.0004,21),error_CZI_r24,'k')
# ax.plot(np.linspace(0.00000,0.0004,21),error_r24,'k')

# ax.plot(np.linspace(0.00000,0.0004,21),error_CZI_r25,'b','-.')
# ax.plot(np.linspace(0.00000,0.0004,21),error_r25,'b','--')
# ax.set_yscale('log')

In [73]:
error_CZI_r14 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_CZI_r14.txt')
error_CZI_r15 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_CZI_r15.txt')
error_CZI_r24 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_CZI_r24.txt')
error_CZI_r25 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_CZI_r25.txt')

error_ICZ_r14 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_ICZ_r14.txt')
error_ICZ_r15 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_ICZ_r15.txt')
error_ICZ_r24 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_ICZ_r24.txt')
error_ICZ_r25 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_ICZ_r25.txt')

error_r14 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_r14.txt')
error_r15 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_r15.txt')
error_r24 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_r24.txt')
error_r25 = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Stray_coupling\error_r25.txt')


In [74]:
fig=plt.figure(figsize=[8.4,4])
ax1 = fig.add_subplot(121)
lw=2.5
ax1.plot( g15_arr ,np.array(error_CZI_r15)*100,'tab:orange',linewidth=lw,label='r15')

ax1.plot( g14_arr ,np.array(error_CZI_r14)*100,'tab:red',linewidth=lw,label='ggggg14')
ax1.plot( g25_arr ,np.array(error_CZI_r25)*100,'tab:blue',linewidth=lw,label='r25')

ax1.plot( g24_arr ,np.array(error_CZI_r24)*100,'tab:green',linewidth=lw,label='r24')

ax1.set_xlim([0,2.4])
# ax1.set_ylim([-0.1,1.7])

ax2 = fig.add_subplot(122)
ax2.plot( g14_arr ,np.array(error_ICZ_r14)*100,'tab:red',linewidth=lw)
ax2.plot( g15_arr ,np.array(error_ICZ_r15)*100,'tab:orange',linewidth=lw)
ax2.plot( g24_arr ,np.array(error_ICZ_r24)*100,'tab:green',linewidth=lw)
ax2.plot( g25_arr ,np.array(error_ICZ_r25)*100,'tab:blue',linewidth=lw)
ax2.set_xlim([0,2.4])
# ax2.set_ylim([-0.1,1.7])

ax1.tick_params(axis='x',which='both',bottom=True,top=True,labelbottom=True,direction='in' )
ax2.tick_params(axis='x',which='both',bottom=True,top=True,labelbottom=True,direction='in' )
ax1.tick_params(axis='y',which='both',left=True,right=True,direction='in' )
ax2.tick_params(axis='y',which='both',left=True,right=True,labelleft=False,direction='in' )

# ax1.set_yscale('log')
# ax2.set_yscale('log')
ax1.tick_params(labelsize=16)
ax2.tick_params(labelsize=16)

ax1.legend(fontsize=16)

In [77]:
fig=plt.figure(figsize=[9,4])
ax1 = fig.add_subplot(121)
lw=2.5
ax1.plot( g15_arr ,np.array(error_CZI_r15)*1,'tab:orange',linewidth=lw,label='r15')

ax1.plot( g14_arr ,np.array(error_CZI_r14)*1,'tab:red',linewidth=lw,label='ggggg14')
ax1.plot( g25_arr ,np.array(error_CZI_r25)*1,'tab:blue',linewidth=lw,label='r25')

ax1.plot( g24_arr ,np.array(error_CZI_r24)*1,'tab:green',linewidth=lw,label='r24')

ax1.set_xlim([0,2.4])
ax1.set_ylim([2.5e-6,0.4e-1])

ax2 = fig.add_subplot(122)
ax2.plot( g14_arr ,np.array(error_ICZ_r14)*1,'tab:red',linewidth=lw)
ax2.plot( g15_arr ,np.array(error_ICZ_r15)*1,'tab:orange',linewidth=lw)
ax2.plot( g24_arr ,np.array(error_ICZ_r24)*1,'tab:green',linewidth=lw)
ax2.plot( g25_arr ,np.array(error_ICZ_r25)*1,'tab:blue',linewidth=lw)
ax2.set_xlim([0,2.4])
ax2.set_ylim([2.5e-6,0.4e-1])

ax1.tick_params(axis='x',which='both',bottom=True,top=True,labelbottom=True,direction='in' )
ax2.tick_params(axis='x',which='both',bottom=True,top=True,labelbottom=True,direction='in' )
ax1.tick_params(axis='y',which='both',left=True,right=True,direction='in' )
ax2.tick_params(axis='y',which='both',left=True,right=True,labelleft=True,direction='in' )

ax1.set_yscale('log')
ax2.set_yscale('log')
ax1.tick_params(labelsize=18)
ax2.tick_params(labelsize=18)

ax2.set(yticklabels=[])

ax1.legend(fontsize=18)

In [60]:
%matplotlib qt